In [2]:
from pathlib import Path
import numpy as np
import csv
import re
import cv2

In [3]:
# 读取相机参数
def read_calib(calib_file_path):
    with open(calib_file_path, 'r') as calib_file:
        calib = {}
        csv_reader = csv.reader(calib_file, delimiter='=')
        for attr, value in csv_reader:
            calib.setdefault(attr, value)
    return calib

def read_pfm(pfm_file_path):
    with open(pfm_file_path, 'rb') as pfm_file:
        header = pfm_file.readline().decode().rstrip()
        channels = 3 if header == 'PF' else 1
        dim_match = re.match(r'^(\d+)\s(\d+)\s$', pfm_file.readline().decode('utf-8'))
        if dim_match:
            width, height = map(int, dim_match.groups())
        else:
            raise Exception("Malformed PFM header.")

        scale = float(pfm_file.readline().decode().rstrip())
        if scale < 0:
            endian = '<'  # littel endian
            scale = -scale
        else:
            endian = '>'  # big endian

        dispariy = np.fromfile(pfm_file, endian + 'f')

    img = np.reshape(dispariy, newshape=(height, width, channels))
    img = np.flipud(img).astype('uint8')
    print(img.max(), img.min())
    show(img, "disparity")
    return dispariy, [(height, width, channels), scale]

def create_depth_map(pfm_file_path, calib=None):
    dispariy, [shape, scale] = read_pfm(pfm_file_path)

    if calib is None:
        raise Exception("Loss calibration information.")
    else:
        fx = float(calib['cam0'].split(' ')[0].lstrip('['))
        base_line = float(calib['baseline'])
        doffs = float(calib['doffs'])
        # scale factor is used here
        # d = bf/(d+ doffs)         doffs就是(x_or-x_ol) 两个相机主点在各自图像坐标系x方向上的坐标差
        depth_map = fx * base_line / (dispariy / scale + doffs)
        depth_map = np.reshape(depth_map, newshape=shape)
        depth_map = np.flipud(depth_map).astype('uint8')
        return depth_map

def show(img, win_name='image'):
    if img is None:
        raise Exception("Can't display an empty image.")
    else:
        cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)
        cv2.imshow(win_name, img)
        cv2.waitKey()
        cv2.destroyWindow(win_name)



In [4]:
pfm_file_dir = Path('demo/pipes-perfect')
calib_file_path = pfm_file_dir.joinpath('calib.txt')
disp_left = pfm_file_dir.joinpath('disp1.pfm')
disparity, [shape, scale] = read_pfm(disp_left)
print(disparity)
disparity = np.reshape(disparity, newshape=shape)
disparity = np.flipud(disparity)
# calibration information
calib = read_calib(calib_file_path)
# create depth map
depth_map_left = create_depth_map(disp_left, calib)
show(depth_map_left, "depth_map")


C:\Users\Administrator\AppData\Local\Temp\ipykernel_43704\2854456553.py:30: RuntimeWarning: invalid value encountered in cast
  img = np.flipud(img).astype('uint8')


255 0
[149.93364 149.91214 149.88821 ... 116.0108  115.87364 115.84978]
255 0


In [1]:
import cv2
import numpy as np

# 加载左图像和视差图
left_img = cv2.imread('vis/heat_map_right.jpg')
disparity = cv2.imread('vis/out_disp.jpg', cv2.IMREAD_GRAYSCALE)
disparity = np.expand_dims(disparity, -1)
print(disparity.shape)
disparity_new = np.float32(disparity.squeeze(-1))
print(disparity_new)

# 创建x和y映射
height, width = disparity_new.shape
x_map = np.float32(np.tile(np.linspace(0, width-1, width), (height, 1)))
y_map = np.float32(np.tile(np.linspace(0, height-1, height), (width, 1)).T)

# 考虑到视差，更新x映射
x_map = x_map - disparity_new

# 使用remap函数进行warp
warped_img = cv2.remap(left_img, x_map, y_map, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

# 保存结果
cv2.imwrite('vis/warped.png', warped_img)

(96, 384, 1)
[[195. 193. 191. ... 188. 186. 186.]
 [194. 193. 191. ... 187. 186. 185.]
 [196. 195. 193. ... 190. 189. 188.]
 ...
 [151. 152. 155. ... 180. 179. 177.]
 [151. 151. 154. ... 178. 176. 175.]
 [151. 151. 154. ... 178. 177. 175.]]


True

In [2]:
from utils import warp
import torch
img_tensor = torch.from_numpy(np.expand_dims(left_img, 0).transpose(0, 3, 1, 2)).float()
print(img_tensor.shape)
print(img_tensor.type())
disparity_tensor = torch.from_numpy(disparity.copy()).unsqueeze(0).permute(0, 3, 1, 2).float()
print(disparity_tensor.type())
print(disparity_tensor)
warped_img_new = warp(img_tensor, disparity_tensor, mode='r2l')
warped_img_new = warped_img_new.view(1,img_tensor.shape[2],img_tensor.shape[3],-1).permute(0,3,1,2).squeeze(0).numpy().astype(np.uint8).transpose(1,2,0)
print(warped_img_new.shape)
# 保存结果
cv2.imwrite('vis/warped_new.png', warped_img_new)

torch.Size([1, 3, 96, 384])
torch.FloatTensor
torch.FloatTensor
tensor([[[[195., 193., 191.,  ..., 188., 186., 186.],
          [194., 193., 191.,  ..., 187., 186., 185.],
          [196., 195., 193.,  ..., 190., 189., 188.],
          ...,
          [151., 152., 155.,  ..., 180., 179., 177.],
          [151., 151., 154.,  ..., 178., 176., 175.],
          [151., 151., 154.,  ..., 178., 177., 175.]]]])
(96, 384, 3)


c:\ProgramData\anaconda3\envs\torch\lib\site-packages\torch\nn\functional.py:4296: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


True